In [ ]:
!pip list

Package                       Version
----------------------------- -------------------
absl-py                       0.12.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.1.0
appdirs                       1.4.4
argcomplete                   1.12.3
argon2-cffi                   20.1.0
arviz                         0.11.2
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-generator               1.10
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.2.0
audioread                     2.1.9
autograd                      1.3
Babel                         2.9.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        4.0.0
blis                          0.4.1
bokeh                         2.3.3
Bottleneck                    1.3.2
branca                        0.4.2
bs4     

In [ ]:
pip install happytransformer

In [ ]:
!nvidia-smi

Sun Aug 15 08:38:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    42W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from happytransformer import HappyQuestionAnswering, QATrainArgs, QATestArgs, QAEvalArgs
import torch
import pandas as pd
import shutil
from google.colab import drive
from psutil import virtual_memory


In [ ]:
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Aug 15 08:38:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
albert_qa = HappyQuestionAnswering("ALBERT","mfeb/albert-xxlarge-v2-squad2")


08/15/2021 08:38:58 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [ ]:
torch.version.cuda

'10.2'

In [ ]:
albert_qa.model

AlbertForQuestionAnswering(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=4096, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((4096,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=4096, out_features=4096, bias=True)
                (key): Linear(in_features=4096, out_features=4096, bias=True)
                (value): Linear(in_features=4096, out_features=40

In [ ]:
albert_qa.model_name

'mfeb/albert-xxlarge-v2-squad2'

In [ ]:
albert_qa.model_type

'ALBERT'

In [ ]:
albert_train_args = QATrainArgs(learning_rate=2e-5, batch_size=1)

In [ ]:
albert_train_args

QATrainArgs(learning_rate=2e-05, num_train_epochs=3.0, weight_decay=0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, save_preprocessed_data=False, save_preprocessed_data_path='', load_preprocessed_data=False, load_preprocessed_data_path='', batch_size=1)

In [ ]:
pd.read_csv("./drive/MyDrive/Research_Transformer/train/train_qa_10k.csv")

,context,question,answer_text,answer_start
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,269
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,207
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,526
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",166
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,276
...,...,...,...,...
9995,The Oklahoman is Oklahoma City's major daily n...,What is the name of The Oklahoman's website?,NewsOK.com,103
9996,The Oklahoman is Oklahoma City's major daily n...,What is Oklahoma Cities newsweekly?,Oklahoma Gazette,150
9997,The Oklahoman is Oklahoma City's major daily n...,What is Oklahoma Cities daily business newspaper?,The Journal Record,338
9998,There are numerous community and international...,Which side is the Black Chronicles Headquarters?,Eastside,156


In [ ]:
!nvidia-smi

Sun Aug 15 18:10:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    42W / 250W |   9863MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
train_output = albert_qa.train("./drive/MyDrive/Research_Transformer/train/train_qa_10k.csv",args=albert_train_args)

08/15/2021 08:39:36 - INFO - happytransformer.happy_transformer -   Preprocessing dataset...
***** Running training *****
  Num examples = 10000
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss
500,0.576200
1000,0.728800
1500,0.584800
2000,0.540500
2500,0.610000
3000,0.694700
3500,0.714000
4000,0.656700
4500,0.678300
5000,0.594100


Saving model checkpoint to /tmp/tmpyy4_dyuh/checkpoint-500
Configuration saved in /tmp/tmpyy4_dyuh/checkpoint-500/config.json
Model weights saved in /tmp/tmpyy4_dyuh/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /tmp/tmpyy4_dyuh/checkpoint-500/tokenizer_config.json
Special tokens file saved in /tmp/tmpyy4_dyuh/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /tmp/tmpyy4_dyuh/checkpoint-1000
Configuration saved in /tmp/tmpyy4_dyuh/checkpoint-1000/config.json
Model weights saved in /tmp/tmpyy4_dyuh/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /tmp/tmpyy4_dyuh/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /tmp/tmpyy4_dyuh/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to /tmp/tmpyy4_dyuh/checkpoint-1500
Configuration saved in /tmp/tmpyy4_dyuh/checkpoint-1500/config.json
Model weights saved in /tmp/tmpyy4_dyuh/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in /tmp/tmpyy4_dyuh/checkpo

In [ ]:
pd.read_csv('./drive/MyDrive/Research_Transformer/val/val_qa_6k.csv')

,context,question,answer_text,answer_start
0,Another possible cause of diarrhea is irritabl...,What is IBS?,irritable bowel syndrome,38
1,Another possible cause of diarrhea is irritabl...,What symptoms do you have with IBS?,abdominal discomfort relieved by defecation an...,98
2,Another possible cause of diarrhea is irritabl...,How can symptoms be managed with IBS?,"a combination of dietary changes, soluble fibe...",302
3,Poverty is a good indicator of the rate of inf...,What is a good indicator of the rate of infect...,Poverty,0
4,Poverty is a good indicator of the rate of inf...,Why does living in poverty raise your risk of ...,from the conditions under which impoverished p...,143
...,...,...,...,...
5995,"BJTs have three terminals, corresponding to th...",How can one find the amount of collector current?,β (common-emitter current gain) times the base...,945
5996,"BJTs have three terminals, corresponding to th...",What is the usual collector current for small-...,greater than 100,1017
5997,"In a FET, the drain-to-source current flows vi...",What determines the conductivity in a FET?,electric field that is produced when a voltage...,156
5998,"In a FET, the drain-to-source current flows vi...",What controls the current between the drain an...,the voltage applied between the gate and source,325


In [ ]:
albert_eval_args = QAEvalArgs()

In [ ]:
albert_eval_args

QAEvalArgs(batch_size=1, save_preprocessed_data=False, save_preprocessed_data_path='', load_preprocessed_data=False, load_preprocessed_data_path='')

In [ ]:
eval_output = albert_qa.eval("./drive/MyDrive/Research_Transformer/val/val_qa_6k.csv", args=albert_eval_args)


PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 6000
  Batch size = 1


In [ ]:
eval_output


EvalResult(loss=1.4828625917434692)

In [ ]:
!mkdir ./drive/MyDrive/Research_Transformer/models

In [ ]:
!mkdir ./drive/MyDrive/Research_Transformer/models/albert

In [ ]:
!mkdir ./drive/MyDrive/Research_Transformer/models/albert/model

In [ ]:
!ls ./drive/MyDrive/Research_Transformer/

models	squad  test  train  val


In [ ]:
!ls ./drive/MyDrive/Research_Transformer/models/

albert


In [ ]:
!ls ./drive/MyDrive/Research_Transformer/models/albert/

model


In [ ]:
albert_eval_loss = {'eval_loss' : [eval_output.loss]}


In [ ]:
albert_eval_loss


{'eval_loss': [1.4828625917434692]}

In [ ]:
albert_eval_loss_df = pd.DataFrame(albert_eval_loss)


In [ ]:
albert_eval_loss_df


,eval_loss
0,1.482863


In [ ]:
albert_eval_loss_df.to_csv(r'./drive/MyDrive/Research_Transformer/models/albert/albert_eval_loss.csv', index=False)


In [ ]:
pd.read_csv('./drive/MyDrive/Research_Transformer/models/albert/albert_eval_loss.csv')


,eval_loss
0,1.482863


In [ ]:
albert_sample_result = albert_qa.answer_question("Macedonia was an ancient kingdom on the periphery of Archaic and Classical Greece, and later the dominant state of Hellenistic Greece. The kingdom was founded and initially ruled by the Argead dynasty, followed by the Antipatrid and Antigonid dynasties. Home to the ancient Macedonians, it originated on the northeastern part of the Greek peninsula. Before the 4th century BC, it was a small kingdom outside of the area dominated by the city-states of Athens, Sparta and Thebes, and briefly subordinate to Achaemenid Persia.",
                         "Who ruled Macedonia?")


In [ ]:
albert_sample_result


[QuestionAnsweringResult(answer=' the Argead dynasty,', score=0.9845701456069946, start=181, end=201)]

In [ ]:
pd.read_csv('./drive/MyDrive/Research_Transformer/test/test_qa_5k.csv')


,context,question,answer_text,answer_start
0,Required attendance at school is 10 years for ...,How long are females required to go to school?,11 years,56
1,Required attendance at school is 10 years for ...,What was the literacy rate on Tuvalu in 2002?,99.0%,112
2,Required attendance at school is 10 years for ...,How many students were in Tuvalu schools in 2010?,"1,918",146
3,Required attendance at school is 10 years for ...,What si the teacher-student ratio for Tuvalu s...,1:18,292
4,Community Training Centres (CTCs) have been es...,What type of school has Tuvalu set up on each ...,Community Training Centres,0
...,...,...,...,...
4995,"In the years that followed, Eisenhower increas...",How many soldiers did Eisenhower ultimately se...,900,106
4996,"In the years that followed, Eisenhower increas...",How many days did the President of South Vietn...,ten,313
4997,"In the years that followed, Eisenhower increas...",Where was a parade held for the President of S...,New York City,416
4998,"In the years that followed, Eisenhower increas...",Who was Secretary of State in 1957?,John Foster Dulles,497


In [ ]:
pd.read_csv('./drive/MyDrive/Research_Transformer/test/test_qa_without_answer_5k.csv')


,context,question
0,Required attendance at school is 10 years for ...,How long are females required to go to school?
1,Required attendance at school is 10 years for ...,What was the literacy rate on Tuvalu in 2002?
2,Required attendance at school is 10 years for ...,How many students were in Tuvalu schools in 2010?
3,Required attendance at school is 10 years for ...,What si the teacher-student ratio for Tuvalu s...
4,Community Training Centres (CTCs) have been es...,What type of school has Tuvalu set up on each ...
...,...,...
4995,"In the years that followed, Eisenhower increas...",How many soldiers did Eisenhower ultimately se...
4996,"In the years that followed, Eisenhower increas...",How many days did the President of South Vietn...
4997,"In the years that followed, Eisenhower increas...",Where was a parade held for the President of S...
4998,"In the years that followed, Eisenhower increas...",Who was Secretary of State in 1957?


In [ ]:
albert_test_args = QATestArgs()


In [ ]:
albert_test_args


QATestArgs(save_preprocessed_data=False, save_preprocessed_data_path='', load_preprocessed_data=False, load_preprocessed_data_path='')

In [ ]:
albert_result = albert_qa.test('./drive/MyDrive/Research_Transformer/test/test_qa_without_answer_5k.csv', args=albert_test_args)


5000it [15:09,  5.50it/s]


In [ ]:
albert_result


[QuestionAnsweringResult(answer=' 11 years', score=0.9999418258666992, start=55, end=64),
 QuestionAnsweringResult(answer=' 99.0%', score=0.99998939037323, start=111, end=117),
 QuestionAnsweringResult(answer=' 1,918', score=0.9999905824661255, start=145, end=151),
 QuestionAnsweringResult(answer=' 1:27.', score=0.9876089692115784, start=385, end=391),
 QuestionAnsweringResult(answer='Community Training Centres', score=0.9999566078186035, start=0, end=26),
 QuestionAnsweringResult(answer=' vocational training', score=0.9998888969421387, start=114, end=134),
 QuestionAnsweringResult(answer=' secondary education.', score=0.9999856948852539, start=230, end=251),
 QuestionAnsweringResult(answer=' Adults', score=0.9999895691871643, start=496, end=503),
 QuestionAnsweringResult(answer=' basic carpentry, gardening and farming, sewing and cooking.', score=0.999479353427887, start=278, end=338),
 QuestionAnsweringResult(answer=' plants and trees', score=0.9991211295127869, start=40, end=57),
 Q

In [ ]:
albert_result[4997].answer

' New York City.'

In [ ]:
albert_result_df = pd.DataFrame([vars(a) for a in albert_result])


In [ ]:
albert_result_df


,answer,score,start,end
0,11 years,0.999942,55,64
1,99.0%,0.999989,111,117
2,"1,918",0.999991,145,151
3,1:27.,0.987609,385,391
4,Community Training Centres,0.999957,0,26
...,...,...,...,...
4995,900,0.999637,105,109
4996,ten,0.998241,312,316
4997,New York City.,0.999985,415,430
4998,John Foster Dulles,0.999990,496,515


In [ ]:
albert_result_df.to_csv(r'./drive/MyDrive/Research_Transformer/models/albert/albert_result.csv', index=False)


In [ ]:
pd.read_csv('./drive/MyDrive/Research_Transformer/models/albert/albert_result.csv')


,answer,score,start,end
0,11 years,0.999942,55,64
1,99.0%,0.999989,111,117
2,"1,918",0.999991,145,151
3,1:27.,0.987609,385,391
4,Community Training Centres,0.999957,0,26
...,...,...,...,...
4995,900,0.999637,105,109
4996,ten,0.998241,312,316
4997,New York City.,0.999985,415,430
4998,John Foster Dulles,0.999990,496,515


In [ ]:
pd.read_csv('./drive/MyDrive/Research_Transformer/test/test_qa_5k.csv')


,context,question,answer_text,answer_start
0,Required attendance at school is 10 years for ...,How long are females required to go to school?,11 years,56
1,Required attendance at school is 10 years for ...,What was the literacy rate on Tuvalu in 2002?,99.0%,112
2,Required attendance at school is 10 years for ...,How many students were in Tuvalu schools in 2010?,"1,918",146
3,Required attendance at school is 10 years for ...,What si the teacher-student ratio for Tuvalu s...,1:18,292
4,Community Training Centres (CTCs) have been es...,What type of school has Tuvalu set up on each ...,Community Training Centres,0
...,...,...,...,...
4995,"In the years that followed, Eisenhower increas...",How many soldiers did Eisenhower ultimately se...,900,106
4996,"In the years that followed, Eisenhower increas...",How many days did the President of South Vietn...,ten,313
4997,"In the years that followed, Eisenhower increas...",Where was a parade held for the President of S...,New York City,416
4998,"In the years that followed, Eisenhower increas...",Who was Secretary of State in 1957?,John Foster Dulles,497


In [ ]:
albert_qa.save('./drive/MyDrive/Research_Transformer/models/albert/model')

Configuration saved in ./drive/MyDrive/Research_Transformer/models/albert/model/config.json
Model weights saved in ./drive/MyDrive/Research_Transformer/models/albert/model/pytorch_model.bin
tokenizer config file saved in ./drive/MyDrive/Research_Transformer/models/albert/model/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/Research_Transformer/models/albert/model/special_tokens_map.json


In [ ]:
!ls -lhac './drive/MyDrive/Research_Transformer/models/albert/model'


total 788M
-rw------- 1 root root  898 Aug 15 19:57 config.json
-rw------- 1 root root 786M Aug 15 19:57 pytorch_model.bin
-rw------- 1 root root  156 Aug 15 19:57 special_tokens_map.json
-rw------- 1 root root 743K Aug 15 19:57 spiece.model
-rw------- 1 root root  660 Aug 15 19:57 tokenizer_config.json
-rw------- 1 root root 1.3M Aug 15 19:57 tokenizer.json
